# UK Online Retail EDA (2009–2011)

This notebook follows the instructor’s **Step-by-Step Tasks**. By request, we start with:
**1) Load & union the data** (no separate project-setup section).

> Edit the config cell below if you want to change file paths or export folders.

### Config (edit as needed)

- Set file paths for the two CSVs.
- Choose where to save figures and small derived tables.

In [ ]:
from pathlib import Path
import warnings, numpy as np, pandas as pd, matplotlib.pyplot as plt

# Display / reproducibility
warnings.filterwarnings("ignore")
np.random.seed(42)
pd.options.display.float_format = lambda x: f"{x:,.2f}"

# ---- EDIT THESE PATHS IF NEEDED ----
DATA_FILES = [
    ("/mnt/data/online_retail_II.xlsx - Year 2009-2010.csv", "2009-2010"),
    ("/mnt/data/online_retail_II.xlsx - Year 2010-2011.csv", "2010-2011"),
]
FIG_DIR  = Path("figures")   # will be created on first save
DATA_DIR = Path("data")      # will be created on first export

print("CSV sources:")
for p, y in DATA_FILES:
    print(f" - {y}: {p}")
print("Figures dir:", FIG_DIR.resolve())
print("Derived-data dir:", DATA_DIR.resolve())

## 1) Load & union the data

- Load both CSVs and **concatenate** into one table.
- Parse `InvoiceDate` to datetime (keeps time).
- Add `YearSheet` column.
- **Checks:** per-file row counts; combined rows; min/max `InvoiceDate`; identical column names.

In [ ]:
def load_and_union(data_files):
    frames, per_file_counts = [], []
    cols_ref = None
    for path, label in data_files:
        df_i = pd.read_csv(path)
        df_i["YearSheet"] = label
        df_i.columns = [c.strip() for c in df_i.columns]  # harmonize
        if cols_ref is None:
            cols_ref = df_i.columns.tolist()
        else:
            assert df_i.columns.tolist() == cols_ref, "Column names mismatch between files."
        per_file_counts.append((label, len(df_i)))
        frames.append(df_i)
    df = pd.concat(frames, ignore_index=True)
    return df, per_file_counts, cols_ref

raw_df, per_file_counts, cols_ref = load_and_union(DATA_FILES)

print("Per-file row counts:", per_file_counts)
print("Combined rows:", len(raw_df))
print("Columns:", cols_ref)

# Parse datetime
raw_df["InvoiceDate"] = pd.to_datetime(raw_df["InvoiceDate"], errors="coerce")
print("Earliest date:", raw_df["InvoiceDate"].min())
print("Latest date:", raw_df["InvoiceDate"].max())

raw_df.head(3)

## 2) Basic cleaning

- Drop **exact duplicate** rows.
- Report % **missing `Customer ID`** (keep for product/country/time EDA; exclude from customer-level views).
- Create: `Is_Return = Quantity < 0`, `Revenue = Quantity * Price`.
- Build **sales subset** for rankings/AOV: `Price > 0` & `Quantity > 0`.
- **Checks:** counts of returns; assert no `Price<=0` or `Qty<=0` in **sales subset**.

In [ ]:
df = raw_df.copy()

# Drop exact duplicates
before = len(df)
df = df.drop_duplicates()
print(f"Dropped duplicates: {before - len(df)} | Remaining: {len(df)}")

# Missing Customer ID
missing_pct = df["Customer ID"].isna().mean() * 100
print(f"%% rows missing Customer ID: {missing_pct:.2f}%%")

# Derived flags
df["Is_Return"] = df["Quantity"] < 0
df["Revenue"]   = df["Quantity"] * df["Price"]

# Sales subset for gross metrics
sales_subset = df[(df["Price"] > 0) & (df["Quantity"] > 0)].copy()

print("Return counts:")
print(df["Is_Return"].value_counts(dropna=False))

assert (sales_subset["Price"] <= 0).sum() == 0, "Found Price<=0 in sales subset."
assert (sales_subset["Quantity"] <= 0).sum() == 0, "Found non-positive Quantity in sales subset."
print("Sales subset rows:", len(sales_subset))

df.head(3)

## 3) Time features

Derive from `InvoiceDate`:
- `Year`, `Quarter`, `Month`, `DayOfWeek` (0=Mon), `Hour`
- `InvoiceDateFloorMonth` (month start) for rollups

**Checks:** value counts for `Hour` and `DayOfWeek`.

In [ ]:
df["Year"]   = df["InvoiceDate"].dt.year
df["Quarter"]= df["InvoiceDate"].dt.quarter
df["Month"]  = df["InvoiceDate"].dt.month
df["DayOfWeek"] = df["InvoiceDate"].dt.dayofweek
df["Hour"]   = df["InvoiceDate"].dt.hour
df["InvoiceDateFloorMonth"] = df["InvoiceDate"].dt.to_period("M").dt.to_timestamp()

sales_subset["InvoiceDateFloorMonth"] = sales_subset["InvoiceDate"].dt.to_period("M").dt.to_timestamp()

print("Hour counts (first 10 hours):")
print(df["Hour"].value_counts().sort_index().head(10))

print("\nDayOfWeek counts (0=Mon):")
print(df["DayOfWeek"].value_counts().sort_index())

## 4) Orders & customers

- Each unique `Invoice` = one **order**.
- Compute **AOV**, **items per order** (total positive qty per invoice), **orders per customer**.
- Tag **New vs Repeat** per `YearSheet` (first purchase **month** logic).
- **Outputs:** table of `n_orders`, `n_customers`, `AOV`, `items_per_order` per `YearSheet`; chart of **New vs Repeat** revenue share.

In [ ]:
# Order metrics (on sales subset)
order_rev = sales_subset.groupby(["YearSheet","Invoice"], as_index=False)["Revenue"].sum()
order_qty = sales_subset.groupby(["YearSheet","Invoice"], as_index=False)["Quantity"].sum()
orders = order_rev.merge(order_qty, on=["YearSheet","Invoice"], how="left")                  .rename(columns={"Revenue":"OrderRevenue","Quantity":"OrderItems"})

summary_orders = orders.groupby("YearSheet").agg(
    n_orders=("Invoice","nunique"),
    AOV=("OrderRevenue","mean"),
    items_per_order=("OrderItems","mean")
).reset_index()

# Customers with IDs
cust_sales = sales_subset.dropna(subset=["Customer ID"]).copy()
cust_sales["Customer ID"] = cust_sales["Customer ID"].astype(int)
n_customers = (cust_sales.groupby("YearSheet")["Customer ID"].nunique()
               .rename("n_customers")).reset_index()

summary_orders = summary_orders.merge(n_customers, on="YearSheet", how="left")
summary_orders[["YearSheet","n_orders","n_customers","AOV","items_per_order"]]

In [ ]:
# New vs Repeat tagging per YearSheet (first purchase month)
def tag_new_repeat(cdf):
    first_month = cdf.groupby("Customer ID")["InvoiceDateFloorMonth"].min().rename("FirstMonth")
    tagged = cdf.join(first_month, on="Customer ID")
    tagged["CustType"] = np.where(tagged["InvoiceDateFloorMonth"]==tagged["FirstMonth"], "New", "Repeat")
    return tagged

cust_month = (cust_sales[["YearSheet","Customer ID","InvoiceDate","InvoiceDateFloorMonth","Revenue"]]).copy()
nr = []
for ys, sub in cust_month.groupby("YearSheet"):
    nr.append(tag_new_repeat(sub).assign(YearSheet=ys))
nr = pd.concat(nr, ignore_index=True)

nr_rev = nr.groupby(["YearSheet","CustType"])["Revenue"].sum().reset_index()
nr_rev["Share"] = nr_rev["Revenue"] / nr_rev.groupby("YearSheet")["Revenue"].transform("sum")
nr_rev

In [ ]:
# Plot: New vs Repeat revenue share
FIG_DIR.mkdir(parents=True, exist_ok=True)
fig, ax = plt.subplots(figsize=(6,4))
for i, ys in enumerate(sorted(nr_rev["YearSheet"].unique())):
    part = nr_rev[nr_rev["YearSheet"]==ys].sort_values("CustType")
    ax.bar([i-0.2, i+0.2], part["Share"].values)
ax.set_xticks(range(len(sorted(nr_rev["YearSheet"].unique()))))
ax.set_xticklabels(sorted(nr_rev["YearSheet"].unique()))
ax.set_ylabel("Revenue Share")
ax.set_title("New vs Repeat Revenue Share by YearSheet")
plt.tight_layout()
out = FIG_DIR/"new_vs_repeat_share.png"
plt.savefig(out); plt.show(); print("Saved:", out)

## 5) Product & country profiles

- **Top 10 products by revenue** (gross: exclude returns when ranking).
- **Top 10 countries by revenue** and **UK vs Rest-of-World** share.
- **Return-prone products**: `return_rate = abs(negative Qty)/total Qty`, **threshold ≥200 units**.

In [ ]:
# Top 10 products (gross revenue)
prod = (sales_subset.groupby(["StockCode","Description"], as_index=False)
        .agg(Revenue=("Revenue","sum"), Quantity=("Quantity","sum")))
top10_products = prod.sort_values("Revenue", ascending=False).head(10).reset_index(drop=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
top10_products.to_csv(DATA_DIR/"top10_products_by_revenue.csv", index=False)
top10_products

In [ ]:
# Top 10 countries (gross revenue)
cty = (sales_subset.groupby("Country", as_index=False)
       .agg(Revenue=("Revenue","sum")))
cty["Share"] = cty["Revenue"]/cty["Revenue"].sum()
top10_countries = cty.sort_values("Revenue", ascending=False).head(10).reset_index(drop=True)
top10_countries.to_csv(DATA_DIR/"top10_countries_by_revenue.csv", index=False)
top10_countries

In [ ]:
# UK vs Rest-of-World chart
FIG_DIR.mkdir(parents=True, exist_ok=True)
uk_rev = cty.loc[cty["Country"]=="United Kingdom","Revenue"].sum()
row_rev = cty.loc[cty["Country"]!="United Kingdom","Revenue"].sum()
fig, ax = plt.subplots(figsize=(5,4))
ax.bar(["United Kingdom","Rest of World"], [uk_rev, row_rev])
ax.set_ylabel("Gross Revenue")
ax.set_title("UK vs Rest-of-World Revenue Share (Gross, Sales Subset)")
plt.tight_layout()
out = FIG_DIR/"uk_vs_row_gross_revenue.png"
plt.savefig(out); plt.show(); print("Saved:", out)

In [ ]:
# Return-prone products (units threshold)
sku_total = df.groupby(["StockCode","Description"])["Quantity"].agg(total_qty=lambda s: s.abs().sum())
sku_ret   = df[df["Quantity"]<0].groupby(["StockCode","Description"])["Quantity"].agg(returns_qty=lambda s: s.abs().sum())
ret = pd.concat([sku_total, sku_ret], axis=1).fillna(0).reset_index()
ret["return_rate"] = np.where(ret["total_qty"]>0, ret["returns_qty"]/ret["total_qty"], 0.0)
ret = ret[ret["total_qty"]>=200].sort_values("return_rate", ascending=False)
ret_top = ret.head(10).reset_index(drop=True)
ret_top.to_csv(DATA_DIR/"return_prone_products.csv", index=False)
ret_top

## 6) Time-series EDA

- **Monthly net revenue** line (returns included as negatives), annotated if needed.
- **Seasonality**: average net revenue by **Month (1–12)**.
- *(Optional)* **Hourly** average revenue (gross, from sales subset).

In [ ]:
# Monthly net revenue
FIG_DIR.mkdir(parents=True, exist_ok=True)
monthly_net = (df.set_index("InvoiceDate").resample("MS")["Revenue"].sum().reset_index())
fig, ax = plt.subplots(figsize=(8,4))
ax.plot(monthly_net["InvoiceDate"], monthly_net["Revenue"])
ax.set_title("Monthly Net Revenue (Returns Included)")
ax.set_xlabel("Month"); ax.set_ylabel("Net Revenue")
plt.tight_layout(); out = FIG_DIR/"monthly_net_revenue.png"
plt.savefig(out); plt.show(); print("Saved:", out)

# Seasonality by month number
monthly_net["MonthNum"] = monthly_net["InvoiceDate"].dt.month
seasonality = monthly_net.groupby("MonthNum")["Revenue"].mean().reset_index()
fig, ax = plt.subplots(figsize=(6,4))
ax.bar(seasonality["MonthNum"], seasonality["Revenue"])
ax.set_title("Average Net Revenue by Month (Seasonality)")
ax.set_xlabel("Month (1–12)"); ax.set_ylabel("Average Net Revenue")
plt.tight_layout(); out = FIG_DIR/"seasonality_avg_net_revenue_by_month.png"
plt.savefig(out); plt.show(); print("Saved:", out)

# Optional hourly pattern (gross, sales subset)
hourly = sales_subset.groupby("Hour")["Revenue"].mean().reset_index()
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(hourly["Hour"], hourly["Revenue"])
ax.set_title("Average Gross Revenue by Hour (Sales Subset)")
ax.set_xlabel("Hour of Day"); ax.set_ylabel("Average Revenue")
plt.tight_layout(); out = FIG_DIR/"hourly_avg_gross_revenue_sales_subset.png"
plt.savefig(out); plt.show(); print("Saved (optional):", out)

## 7) Returns analysis

- **Overall return rate** (units): `sum(negative Qty)/sum(abs(Qty))`.
- **Revenue impact**: `sum(negative Revenue)` vs **net** revenue.
- Breakouts **by Country** and for **Top 10 products**.

In [ ]:
neg_units = df.loc[df["Quantity"]<0,"Quantity"].abs().sum()
tot_units = df["Quantity"].abs().sum()
overall_return_rate = (neg_units/tot_units) if tot_units>0 else np.nan
neg_revenue = df.loc[df["Revenue"]<0,"Revenue"].sum()
net_revenue = df["Revenue"].sum()
print(f"Overall return rate (units): {overall_return_rate:.4f}")
print(f"Negative revenue (returns): {neg_revenue:,.2f}")
print(f"Net revenue: {net_revenue:,.2f}")

# By country
by_country = df.groupby("Country").agg(
    neg_units=("Quantity", lambda s: s[s<0].abs().sum()),
    total_units=("Quantity", lambda s: s.abs().sum()),
    neg_revenue=("Revenue", lambda s: s[s<0].sum()),
    net_revenue=("Revenue","sum")
).reset_index()
by_country["return_rate"] = np.where(by_country["total_units"]>0, by_country["neg_units"]/by_country["total_units"], np.nan)
by_country = by_country.sort_values("return_rate", ascending=False)
DATA_DIR.mkdir(parents=True, exist_ok=True)
by_country.to_csv(DATA_DIR/"returns_by_country.csv", index=False)
by_country.head(15)

In [ ]:
# For the previously computed top-10 products
top10_codes = set(top10_products["StockCode"].tolist())
by_product = df[df["StockCode"].isin(top10_codes)].groupby(["StockCode","Description"]).agg(
    neg_units=("Quantity", lambda s: s[s<0].abs().sum()),
    total_units=("Quantity", lambda s: s.abs().sum()),
    neg_revenue=("Revenue", lambda s: s[s<0].sum()),
    net_revenue=("Revenue","sum")
).reset_index()
by_product["return_rate"] = np.where(by_product["total_units"]>0, by_product["neg_units"]/by_product["total_units"], np.nan)
by_product = by_product.sort_values("return_rate", ascending=False)
by_product.to_csv(DATA_DIR/"returns_for_top10_products.csv", index=False)
by_product

## 8) Customer-level snapshots (RFM-lite)

Where `Customer ID` is present:
- **R:** days since last purchase (at dataset end)
- **F:** number of invoices
- **M:** total net revenue

In [ ]:
cust_df = df.dropna(subset=["Customer ID"]).copy()
cust_df["Customer ID"] = cust_df["Customer ID"].astype(int)

dataset_end = df["InvoiceDate"].max()
last_purchase = cust_df.groupby("Customer ID")["InvoiceDate"].max()
recency_days = (dataset_end - last_purchase).dt.days.rename("RecencyDays")
frequency = cust_df.groupby("Customer ID")["Invoice"].nunique().rename("Frequency")
monetary = cust_df.groupby("Customer ID")["Revenue"].sum().rename("Monetary")
rfm = pd.concat([recency_days, frequency, monetary], axis=1).reset_index()

DATA_DIR.mkdir(parents=True, exist_ok=True)
rfm.to_csv(DATA_DIR/"rfm_snapshot.csv", index=False)

# Histograms (separate plots)
FIG_DIR.mkdir(parents=True, exist_ok=True)
fig, ax = plt.subplots(figsize=(6,4))
ax.hist(rfm["RecencyDays"].dropna(), bins=30)
ax.set_title("Recency (days since last purchase)"); ax.set_xlabel("Days"); ax.set_ylabel("Customers")
plt.tight_layout(); out = FIG_DIR/"rfm_hist_recency.png"; plt.savefig(out); plt.show(); print("Saved:", out)

fig, ax = plt.subplots(figsize=(6,4))
ax.hist(rfm["Frequency"].dropna(), bins=30)
ax.set_title("Frequency (# of invoices)"); ax.set_xlabel("Invoices"); ax.set_ylabel("Customers")
plt.tight_layout(); out = FIG_DIR/"rfm_hist_frequency.png"; plt.savefig(out); plt.show(); print("Saved:", out)

fig, ax = plt.subplots(figsize=(6,4))
ax.hist(rfm["Monetary"].dropna(), bins=30)
ax.set_title("Monetary (net revenue per customer)"); ax.set_xlabel("Revenue"); ax.set_ylabel("Customers")
plt.tight_layout(); out = FIG_DIR/"rfm_hist_monetary.png"; plt.savefig(out); plt.show(); print("Saved:", out)

print("RFM five-number summaries:")
rfm.describe(percentiles=[0.25,0.5,0.75]).T[["min","25%","50%","75%","max"]]

## 9) Synthesis & recommendations (for your manuscript)

Write **5–7 insights**, each supported by a figure/table above, and propose **3 actions** the retailer can test next month.
- Example actions: peak-hour staffing/stocking, SKU-specific return reduction, segmented bundles for repeaters/new customers.